# Notebook 14: Scaled Thermal Convection

This notebook demonstrates a complete thermal convection problem with non-dimensional scaling. We'll solve a coupled Stokes-temperature system with realistic mantle convection parameters.

In this notebook you'll learn:
- Setting up coupled Stokes and temperature equations
- Computing buoyancy forces from temperature
- Using realistic mantle convection parameters
- Working with the units system for dimensional/non-dimensional conversion
- Understanding the Rayleigh number

In [1]:
import nest_asyncio
nest_asyncio.apply()

import os
os.environ["SYMPY_USE_CACHE"] = "no"

import numpy as np
import sympy
import underworld3 as uw

## Problem Setup: Steady-State Thermal Convection

We'll solve thermal convection in a square box with:
- Hot bottom boundary (driving convection)
- Cold top boundary
- Free-slip walls
- Incompressible Stokes flow

The equations are:
- **Stokes**: ∇·σ = -ρ₀αgT (buoyancy from temperature)
- **Energy**: u·∇T = κ∇²T (steady-state, advection = diffusion)

## Reference Quantities and Physical Parameters

Set up realistic mantle convection parameters using lazy evaluation:

In [2]:
# Reset and configure model with reference quantities
uw.reset_default_model()
model = uw.get_default_model()

model.set_reference_quantities(
    domain_depth=uw.quantity(2900, "km"),
    plate_velocity=uw.quantity(5, "cm/year"),
    mantle_viscosity=uw.quantity(1e21, "Pa*s"),
    temperature_difference=uw.quantity(3000, "K"),
)

uw.use_nondimensional_scaling(True)


# Physical parameters - use uw.expression() for lazy evaluation
# Expressions update automatically when their values change
rho0 = uw.expression(r"\rho_0", sym=uw.quantity(3300, "kg/m^3"))
alpha = uw.expression(r"\alpha", sym=uw.quantity(3e-5, "1/K"))
g = uw.expression("g", sym=uw.quantity(9.8, "m/s^2"))
kappa = uw.expression(r"\kappa", sym=uw.quantity(1e-6, "m^2/s"))
eta0 = uw.expression(r"\eta_0", sym=uw.quantity(1e21, "Pa*s"))
DeltaT = uw.expression(r"\Delta T", sym=uw.quantity(3000, "K"))
L = uw.expression("L", sym=uw.quantity(2900, "km"))

# Display
uw.pprint(f"ρ₀ = {rho0}")
uw.pprint(f"α = {alpha}")
uw.pprint(f"g = {g}")
uw.pprint(f"κ = {kappa}")
uw.pprint(f"η = {eta0}")
uw.pprint(f"ΔT = {DeltaT}")
uw.pprint(f"L = {L}")

ρ₀ = 3300.0 kilogram / meter ** 3
α = 3e-05 1 / kelvin
g = 9.8 meter / second ** 2
κ = 1e-06 meter ** 2 / second
η = 1e+21 pascal * second
ΔT = 3000.0 kelvin
L = 2900.0 kilometer


## Compute Rayleigh Number

The Rayleigh number Ra characterizes the vigor of convection:

Ra = (ρ₀ α g ΔT L³) / (η κ)

Higher Ra means more vigorous convection.

In [3]:
# Rayleigh number: Ra = (ρ₀ α g ΔT L³) / (η κ)
# Use .quantity property to get numeric values for arithmetic with proper unit tracking
# Ra_value = (rho0.quantity * alpha.quantity * g.quantity * 
#             DeltaT.quantity * L.quantity**3) / (eta0.quantity * kappa.quantity)

# # Simplify complex mixed units (km vs m) to reveal dimensionless result
# Ra = Ra_value.to_reduced_units()

# uw.pprint(f"Rayleigh number: {Ra.magnitude:.2e}")
# uw.pprint(f"  Units: {Ra.units}")  # Should show "dimensionless"

In [4]:
Ra_s = (rho0 * alpha * g * DeltaT * L**3) / (eta0 * kappa)
uw.function.evaluate(Ra_s, np.array([[0.0,0.0]]))

UWQuantity([[[70986623.4]]], 'dimensionless')

## Create Mesh and Variables

Set up the computational domain (non-dimensional coordinates 0 to 1):

In [5]:
# Create mesh
resolution = 32

ymax = (uw.quantity(2900, "km"))
xmax = (uw.quantity(2900, "km"))


mesh = uw.meshing.UnstructuredSimplexBox(
    cellSize=0.05*xmax,
    minCoords=(0.0, 0.0),
    maxCoords=(xmax, ymax),
    qdegree=3,
)


# Create variables with physical units
v = uw.discretisation.MeshVariable("v", mesh, 2, degree=2, units="m/s")
p = uw.discretisation.MeshVariable("p", mesh, 1, degree=1, units="Pa")
T = uw.discretisation.MeshVariable("T", mesh, 1, degree=2, units="K")

# Get coordinate symbols
x, y = mesh.X

uw.pprint(f"Mesh: {resolution}×{resolution} elements")
uw.pprint(f"Variables created with units: v [{uw.get_units(v)}], p [{uw.get_units(p)}], T [{uw.get_units(T)}]")

Mesh: 32×32 elements
Variables created with units: v [meter / second], p [pascal], T [kelvin]


## Initialize Temperature Field

Set up initial temperature with perturbation to seed convection:

In [6]:
# Initial temperature: hot bottom (1), cold top (0), with perturbation
# T = (1 - y) + 0.1*cos(2πx)*sin(πy)
# This is in non-dimensional form [0,1]

T_init_nd = (1.0 - y) + 0.1 * sympy.cos(2 * sympy.pi * x) * sympy.sin(sympy.pi * y)

# Convert to dimensional by multiplying by temperature scale
# DeltaT.quantity gives us the numeric UWQuantity value

with uw.mpi.selective_ranks([0]):
    T.array[...] = uw.function.evaluate(T_init_nd, T.coords)

uw.pprint(f"Initial T range: [{T.min()}, {T.max()}]")

Initial T range: [0.0 kelvin, 3000.0 kelvin]


## Set Up Stokes Solver

Configure Stokes flow with temperature-dependent buoyancy:

In [7]:
# Create Stokes solver
stokes = uw.systems.Stokes(mesh, velocityField=v, pressureField=p)
stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel

# Expressions can be used directly as quantities (they inherit from UWQuantity)
stokes.constitutive_model.Parameters.shear_viscosity_0 = eta0

stokes.petsc_options.setValue("ksp_monitor", None)
stokes.petsc_options.setValue("snes_monitor", None)


# Buoyancy force: F = -ρ₀αgT (dimensionless form using Ra)
unit_vertical = mesh.CoordinateSystem.unit_e_1

stokes.bodyforce = -g * rho0 * alpha * T[0] * unit_vertical

# Free-slip boundary conditions
stokes.add_dirichlet_bc((0.0, sympy.oo), "Left")   # u_x = 0 on left
stokes.add_dirichlet_bc((0.0, sympy.oo), "Right")  # u_x = 0 on right
stokes.add_dirichlet_bc((sympy.oo, 0.0), "Top")    # u_y = 0 on top
stokes.add_dirichlet_bc((sympy.oo, 0.0), "Bottom") # u_y = 0 on bottom

uw.pprint("Stokes solver configured with free-slip BCs")

Stokes solver configured with free-slip BCs


In [8]:
stokes.solve()
# stokes.estimate_dt().to("year")
# stokes.get_snes_diagnostics()

  0 SNES Function norm 2.341497410341e+02
    Residual norms for Solver_22_ solve.
    0 KSP Residual norm 5.481310760777e+04
    1 KSP Residual norm 1.535406921379e+00
  1 SNES Function norm 1.079812869075e-03


In [9]:
stokes.estimate_dt().to('yr')

UWQuantity(48399.370066784984, 'year')

## Set Up Temperature Solver

Configure advection-diffusion for temperature evolution:

In [10]:
# Create advection-diffusion solver
adv_diff = uw.systems.AdvDiffusionSLCN(
    mesh,
    u_Field=T,
    V_fn=v.sym,
)

# Constitutive model for diffusion
adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel

# Expressions can be used directly as quantities (they inherit from UWQuantity)
adv_diff.constitutive_model.Parameters. diffusivity = kappa

# Temperature boundary conditions - expressions work directly
adv_diff.add_dirichlet_bc(DeltaT, "Bottom")  # Hot bottom
adv_diff.add_dirichlet_bc(uw.quantity(0, "K"), "Top")  # Cold top

uw.pprint("Temperature solver configured")

Temperature solver configured


In [15]:
adv_diff.estimate_dt().to("year")

UWQuantity(25336322569.621605, 'year')

## Solve Coupled System Iteratively

Since temperature affects velocity (buoyancy) and velocity affects temperature (advection), we iterate:

1. Solve Stokes for velocity (given temperature)
2. Solve temperature equation (given velocity)
3. Repeat until convergence

In [14]:
stokes.estimate_dt().to("year")

UWQuantity(48399.370066784984, 'year')

In [16]:
# Iterative solution
n_iterations = 20

for iteration in range(n_iterations):
    # Solve Stokes
    stokes.solve(zero_init_guess=False)
    
    # Solve temperature (use large timestep for steady-state)
    adv_diff.solve(timestep=uw.quantity(50000, "year"))
    
    # Monitor convergence
    if iteration % 5 == 0:
        v_max = v.max()
        uw.pprint(f"Iteration {iteration}: max velocity = {v_max}")

uw.pprint("\nConverged solution obtained")

  0 SNES Function norm 2.341497410341e+02
  0 SNES Function norm 1.079812869075e-03
Iteration 0: max velocity = (UWQuantity(1.984019208701762e-08, 'meter / second'), UWQuantity(3.960787419397947e-08, 'meter / second'))
  0 SNES Function norm 2.331300760894e+02
  0 SNES Function norm 2.102106109614e+01
    Residual norms for Solver_22_ solve.
    0 KSP Residual norm 1.389197199581e+03
    1 KSP Residual norm 4.035352019620e-02
  1 SNES Function norm 7.348751585361e-04
  0 SNES Function norm 2.336552439870e+02
  0 SNES Function norm 4.459747515515e+00
    Residual norms for Solver_22_ solve.
    0 KSP Residual norm 6.431424002302e+02
    1 KSP Residual norm 1.429723782262e-02
  1 SNES Function norm 7.878284362821e-05
  0 SNES Function norm 2.347843155878e+02
  0 SNES Function norm 3.818567866890e+00
    Residual norms for Solver_22_ solve.
    0 KSP Residual norm 7.091970304915e+02
    1 KSP Residual norm 1.938122776467e-02
  1 SNES Function norm 8.965084106550e-05
  0 SNES Function norm

## Examine Results in Dimensional Units

Let's look at the solution in physical units:

In [17]:
# Velocity magnitude
v_max_components = v.max()
v_max_value = max(v_max_components)

# Convert to common geological units
v_cm_year = v_max_value.to("cm/year")
v_mm_year = v_max_value.to("mm/year")

uw.pprint(f"Maximum velocity: {v_cm_year:.2f} = {v_mm_year:.2f}")

# Temperature range
T_min = T.min()
T_max = T.max()

uw.pprint(f"Temperature range: {T_min:.1f} to {T_max:.1f}")

Maximum velocity: 4.65 centimeter / year = 46.54 millimeter / year
Temperature range: 0.0 kelvin to 3000.0 kelvin


The convection velocities are on the order of centimeters per year, similar to tectonic plate velocities!

## Compute Strain Rate

Calculate strain rate magnitude from velocity field:

In [18]:
# Strain rate tensor: ε̇ᵢⱼ = ½(∂vᵢ/∂xⱼ + ∂vⱼ/∂xᵢ)
strain_rate_var = uw.discretisation.MeshVariable("eps", mesh, 1, degree=1, units="1/s")

# Compute second invariant: √(½ε̇ᵢⱼε̇ᵢⱼ)
eps = sympy.Matrix([[v.sym[0].diff(x), 0.5*(v.sym[0].diff(y) + v.sym[1].diff(x))],
                    [0.5*(v.sym[0].diff(y) + v.sym[1].diff(x)), v.sym[1].diff(y)]])

strain_rate_expr = sympy.sqrt(0.5 * (eps[0,0]**2 + eps[1,1]**2 + 2*eps[0,1]**2))

# Project to mesh variable
proj = uw.systems.Projection(mesh, strain_rate_var)
proj.uw_function = strain_rate_expr
proj.solve()

# Display in geological units
eps_max = strain_rate_var.max()
eps_per_year = eps_max.to("1/year")

uw.pprint(f"Maximum strain rate: {eps_max:.3e} = {eps_per_year:.3e}")

Maximum strain rate: 2.041e-15 1 / second = 6.440e-08 1 / year


## Visualize Results (Optional)

Plot temperature field and velocity vectors:

In [19]:
# visualise it


if uw.mpi.size == 1:
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["P"] = vis.scalar_fn_to_pv_points(pvmesh, p.sym)
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v.sym)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, T.sym)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(T)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, T.sym)

    skip = 1
    points = np.zeros((mesh._centroids[::skip].shape[0], 3))
    points[:, 0] = mesh._centroids[::skip, 0]
    points[:, 1] = mesh._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)

    pvstream = pvmesh.streamlines_from_source(
        point_cloud,
        vectors="V",
        integration_direction="both",
        integrator_type=45,
        surface_streamlines=True,
        initial_step_length=0.01,
        max_time=1.0,
        max_steps=500,
    )

    pl = pv.Plotter(window_size=(750, 750))

    pl.add_mesh(
        pvmesh,
        cmap="RdBu_r",
        edge_color="Grey",
        edge_opacity=0.33,
        scalars="T",
        show_edges=True,
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=True,
    )

    pl.add_mesh(
        pvmesh_t,
        cmap="RdBu_r",
        edge_color="Grey",
        edge_opacity=0.33,
        scalars="T",
        show_edges=True,
        use_transparency=False,
        opacity=1.0,
        show_scalar_bar=True,
    )


    # pl.add_mesh(
    #     pvstream,
    #     opacity=0.33,
    #     show_scalar_bar=False,
    #     cmap="Greens",
    #     render_lines_as_tubes=False,
        
    # )

    pl.export_html("html5/scaled_convection_plot.html")
    # pl.show(cpos="xy", jupyter_backend="trame")
    pl.show()

Widget(value='<iframe src="http://localhost:58437/index.html?ui=P_0x35a5b75f0_0&reconnect=auto" class="pyvista…

## Summary

We successfully solved a thermal convection problem with realistic physical parameters using lazy evaluation:

**Setup:**
- Defined mantle reference quantities (depth, velocity, viscosity, temperature)
- Created variables with physical units (m/s, Pa, K)
- Used **`uw.expression()`** for **lazy evaluation** of physical parameters

**Physics:**
- Rayleigh number Ra ≈ 7×10⁷ (vigorous convection)
- Coupled Stokes flow with temperature-dependent buoyancy
- Steady-state advection-diffusion balance

**Results:**
- Velocities ~cm/year (realistic for mantle convection)
- Strain rates ~10⁻¹⁵ /s (typical geological rates)
- Temperature structure shows convection cells

### Key Insights

1. **Lazy Evaluation**: Use `uw.expression(name, sym=uw.quantity(value, units))` for parameters that can be updated dynamically
2. **Dual Nature**: Expressions serve TWO purposes:
   - **Symbolic variables** for PDEs: `rho0 * T` returns SymPy expression
   - **Numeric quantities** for arithmetic: `rho0.quantity` returns UWQuantity with units
3. **Direct Use**: Expressions inherit from UWQuantity, so can be used directly in solver parameters
4. **Unit Objects**: All units are Pint Unit objects (never strings) - `uw.get_units(qty)` returns `<Unit('meter/second')>`
5. **Unit Simplification**: Use `.to_reduced_units()` to simplify complex unit expressions

### Expression Usage Patterns

```python
# 1. Define expressions with units (lazy evaluation)
rho0 = uw.expression(r"\\rho_0", sym=uw.quantity(3300, "kg/m^3"))
alpha = uw.expression(r"\\alpha", sym=uw.quantity(3e-5, "1/K"))

# 2a. Use in PDEs (symbolic - automatic substitution)
buoyancy_force = rho0 * alpha * g * T  # Returns SymPy expression

# 2b. Use directly as quantities (they inherit from UWQuantity)
stokes.constitutive_model.Parameters.viscosity = eta0  # Works directly!

# 2c. Numeric arithmetic with units (for calculations)  
Ra = (rho0.quantity * alpha.quantity * g.quantity * 
      DeltaT.quantity * L.quantity**3) / (eta0.quantity * kappa.quantity)
# Returns UWQuantity with properly combined units

# 3. Simplify complex units
Ra_clean = Ra.to_reduced_units()  # "kg·km³/m⁴/Pa/s²" → "dimensionless"

# 4. Check units (returns Pint Unit object, NOT string)
units = uw.get_units(rho0)  # <Unit('kilogram / meter ** 3')>
print(type(units))  # <class 'pint.Unit'>
```

### Units Architecture Principle

**CRITICAL**: Underworld3 follows the principle **"String input, Pint object storage/output"**

- ✅ **INPUT**: Accept strings for user convenience: `uw.quantity(5, "cm/year")`
- ✅ **STORAGE**: Store as Pint objects internally for dimensional analysis
- ✅ **OUTPUT**: Return Pint Unit objects (NOT strings): `qty.units` → `<Unit('...')>`

This enables:
- Dimensional analysis and compatibility checking
- Unit arithmetic: `velocity.units / time.units` → `<Unit('meter / second ** 2')>`
- Type safety: Pint objects have methods like `.to()`, `.dimensionality`

### Units API Reference

**UWQuantity/UWexpression methods**:
- `.quantity` - Get numeric UWQuantity value (for expressions)
- `.units` - Get Pint Unit object (NOT string!)
- `.to(target_units)` - Convert to specific units: `qty.to("cm/year")`
- `.to_base_units()` - Convert to SI base units
- `.to_reduced_units()` - Simplify unit expressions by canceling factors
- `.to_compact()` - Automatically select readable units

**Global functions**:
- `uw.get_units(expr)` - Get Pint Unit object (NOT string!)
- `uw.quantity(value, "units")` - Create quantity (string input, Pint storage)
- `uw.expression(name, sym=qty)` - Create lazy-evaluated expression

### What's Next?

To extend this example:
- Add time evolution (transient convection)
- Include temperature-dependent viscosity: `eta0.sym = new_value` (lazy update!)
- Add internal heating (radioactive decay)
- Compute Nusselt number (heat flux efficiency)
- Explore parameter sensitivity by updating expression values dynamically